In [1]:
!pip install nltk
!pip install pandas
!pip install numpy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install ipython-autotime

%load_ext autotime

     |████████████████████████████████| 1.5 MB 4.4 MB/s            
     |████████████████████████████████| 748 kB 60.7 MB/s            
     |████████████████████████████████| 97 kB 8.1 MB/s             


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


time: 670 µs (started: 2022-02-01 08:36:45 +00:00)


In [29]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

In [14]:
paths = []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+"20_newsgroups"+'/comp.graphics'):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)


In [15]:
paths[0]

'/home/jovyan/binder/20_newsgroups/comp.graphics/39673'

In [16]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

In [17]:
def convert_lower_case(data):
    return np.char.lower(data)

In [18]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [19]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [20]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [21]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [22]:
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

In [23]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

In [24]:
def preprocess(data, query):
    if not query:
        data = remove_header(data)        
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    data = stemming(data)
    return data

In [31]:
postings = pd.DataFrame()
frequency = pd.DataFrame()
doc = 0

for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)
    if doc%100 == 0:
        print(doc)

    tokens = word_tokenize(str(preprocessed_text))
    
    pos = 0
    for token in tokens:
        if token in postings:
            p = postings[token][0]            

            k = [a[0] for a in p]
            if doc in k:
                for a in p:
                    if a[0] == doc:
                        a[1].add(pos)
            else:
                p.append([doc,{pos}])
                frequency[token][0] += 1
        else:
            postings.insert(value=[[[doc, {pos}]]], loc=0, column=token)
            frequency.insert(value=[1], loc=0, column=token)

        pos += 1
    doc += 1

0
100
200
No Header
300
No Header
400


In [32]:
postings

,cold,stink,amsterdam,uva,fwi,aio,wijkstra,marcel,noturn,ppmtopgm,...,nine,one,articl,in,wrote,ch,cern,vxcrna,filip,vinci
0,"[[405, {264}]]","[[405, {260}]]","[[405, {256}]]","[[405, {246}]]","[[405, {245}]]","[[405, {243}]]","[[405, {242, 244}]]","[[405, {240, 241}]]","[[405, {99, 132, 125}]]","[[405, {90}]]",...,"[[0, {37, 38, 70, 71, 9, 10, 16, 51, 83, 22, 3...","[[0, {8, 520, 652, 13, 15, 409, 667, 36, 439, ...","[[0, {35, 68, 7}], [1, {32, 40}], [4, {1}], [8...","[[0, {641, 67, 6, 488, 393, 560, 1041, 754, 10...","[[0, {5}], [16, {15, 7}], [17, {73}], [23, {55...","[[0, {4}], [8, {68, 102, 138, 78, 113, 149, 91...","[[0, {3}]]","[[0, {2}]]","[[0, {1}]]","[[0, {0}]]"


In [33]:
frequency

,cold,stink,amsterdam,uva,fwi,aio,wijkstra,marcel,noturn,ppmtopgm,...,nine,one,articl,in,wrote,ch,cern,vxcrna,filip,vinci
0,1,1,1,1,1,1,1,1,1,1,...,176,280,121,311,30,14,1,1,1,1


In [34]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",frequency[preprocessed_word][0])
    print("Postings List:",postings[preprocessed_word][0])

In [35]:
def get_positions(posting_values, doc):
    for posting_value in posting_values:
        if posting_value[0] == doc:
            return posting_value[1]
    return {}

In [36]:
def gen_init_set_matchings(word):
    init = []
    word_postings = postings[word][0]
    for word_posting in word_postings:
        for positions in word_posting[1]:
            init.append((word_posting[0], positions))
    return init

In [37]:
def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]

        count = 0

        for k in b:
            pos = pos+1
            k_pos = postings[k][0]
            docs_list = [z[0] for z in k_pos]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)

In [38]:
def run_query(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        print("Total Document Mathces", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(init_word)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    return total_matched_docs

In [39]:
def print_file(file):
    out_file = open(paths[file], 'r', encoding='cp1250')
    out_text = out_file.read()
    print(out_text)

In [40]:
get_word_postings("welcome")

welcom
Frequency: 9
Postings List: [[0, {631}], [73, {26, 6508, 6583}], [189, {91}], [268, {88}], [270, {213}], [308, {61}], [344, {40}], [371, {196}], [399, {281}]]


In [41]:
query = "not in"

In [42]:
lists = run_query(query)

not in
['not', 'in']
Total Document Matches: {97, 324, 199, 8, 73, 201, 75, 80, 337, 371, 214, 215, 185, 90}
